<a href="https://colab.research.google.com/github/aki-yoshimoto/qmacro/blob/main/Project2_in_googleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')
#data PWT 10.01 in https://www.rug.nl/ggdc/productivity/pwt/ 1950-2019

#project2 conuntry ”France”, ”Germany”, ”Canada”, ”Italy”, ”Japan”, ”United Kingdom”, ”United States”, 1995-2019

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France", "Germany","Canada","Italy","Japan",
                                             "United Kingdom", "United States"])][
                                                 ['year', 'country', 'rgdpna', 
                                                  'rkna','pop', 'emp','avh', 'labsh','rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()


# Calculate additional columns
# data['labsh'] TFP share
data['alpha'] = 1 - data['labsh'] # Capital share
data['labor'] = data['emp'] * data['avh']  # L
data['y_l'] = data['rgdpna'] / data['labor'] # GDP per worker Y/L
data['y_l'] = np.log(data['y_l'])  # ln(Y/L)
data['k_l'] = data['rkna'] / data['labor'] # Capital K/L
data['k_l'] = np.log(data['k_l'])  # ln(K/L)
# ln(TFP) = ln(Y/L) - alpha * ln(K/L)
data['tfp_term'] = data['y_l'] - data['alpha'] * data['k_l'] #tfp
data['cap_term'] = data['alpha'] * data['k_l']  # cap


# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('country')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_l'].diff() * 100)  # Growth rate 
data['tfp'] = (grouped_data['tfp_term'].diff() * 100) #TFP Growth
data['cap'] = (grouped_data['cap_term'].diff() * 100) #Capital Deepning

# Remove missing values
data = data.dropna()

# Calculate summary statistics #
summary = data.groupby('country').agg({'g': 'mean',
                                       'alpha': "mean",
                                       'labsh':"mean",
                                       'tfp':'mean',
                                       'cap':'mean'})


# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['TFP Growth'] = summary['tfp']
summary['Capital Deepning']=summary['cap']
summary['TFP Share'] = summary['labsh']
summary['Capital Share'] = summary['alpha']

# Remove unnecessary columns
summary = summary.drop(['g', 'alpha', 'labsh', 'tfp', 'cap'], axis=1)

# Print output
print(summary)








                Growth Rate  TFP Growth  Capital Deepning  TFP Share  \
country                                                                
Canada             1.117384    1.677696         -0.560312   0.653810   
France             1.087678    0.247013          0.840665   0.618806   
Germany            1.073874    1.409886         -0.336012   0.629270   
Italy              0.285631   -0.256090          0.541721   0.515165   
Japan              1.221666    2.118842         -0.897175   0.569399   
United Kingdom     1.149943   -1.598219          2.748162   0.587007   
United States      1.658237    1.355163          0.303074   0.606740   

                Capital Share  
country                        
Canada               0.346190  
France               0.381194  
Germany              0.370730  
Italy                0.484835  
Japan                0.430601  
United Kingdom       0.412993  
United States        0.393260  
